In [2]:
#importing necessary libraries
import networkx as nx
import pandas as pd
import spacy

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Twitter Dataset
Can be found here: https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter

The dataset is a CSV, where each row is a tweet. The different columns are described below. Every conversation included has at least one request from a consumer and at least one response from a company. Which user IDs are company user IDs can be calculated using the inbound field.

tweet_id: A unique, anonymized ID for the Tweet. Referenced by response_tweet_id and in_response_to_tweet_id.

author_id:
A unique, anonymized user ID. @s in the dataset have been replaced with their associated anonymized user ID.

inbound:
Whether the tweet is "inbound" to a company doing customer support on Twitter. This feature is useful when re-organizing data for training conversational models.

created_at:
Date and time when the tweet was sent.

text:
Tweet content. Sensitive information like phone numbers and email addresses are replaced with mask values like __email__.

response_tweet_id:
IDs of tweets that are responses to this tweet, comma-separated.

in_response_to_tweet_id:
ID of the tweet this tweet is in response to, if any.



In [3]:
complete_tweets_dataframe = pd.read_csv('/content/drive/MyDrive/twcs.csv') #import dataset from kaggle

In [12]:
len(df)-6505

2805269

In [4]:
#the tweets will be converted to a text file to create a knowledge graph
#the knowledge graph library can only take files upto 1,000,000 characters
#hence we're dropping rows from the table
complete_tweets_dataframe.drop(complete_tweets_dataframe.tail(2805269).index, inplace = True)

In [5]:
len(complete_tweets_dataframe) #this size will cause no problems during training

6505

### Methodology

Proposed Arch: the model is given the tweets but the architecture used here is the one mentioned in [the deck](https://docs.google.com/presentation/d/1ByRYqVdoMTBallftR0YQXJUkgZUXom17YBkiMkdFWHE/edit?usp=sharing) and shown [here](https://whimsical.com/team-novice-current-architecture-MuNsYH5WVJLZDwCjohaeew).

We will show the performance of both models when all 6505 tweets are given for training

# Take top 'n' rows from Tweets Table as Input

In [ ]:
def extract_top_rows(df, num_rows):
    """
    Extract the specified number of rows from the top of a DataFrame.

    Parameters:
    df (pandas.DataFrame): The input DataFrame.
    num_rows (int): The number of rows to extract.

    Returns:
    pandas.DataFrame: A new DataFrame containing the extracted rows.
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame")

    if not isinstance(num_rows, int) or num_rows <= 0:
        raise ValueError("num_rows must be a positive integer")

    if num_rows > len(df):
        print(f"Warning: Requested {num_rows} rows, but DataFrame only has {len(df)} rows. Returning all rows.")
        return df.copy()

    return df.head(num_rows).copy()



# Vector DB Preparation

Proposed Arch: We will convert the given tweets data into organised tweet threads. Then we'll create a knowledge graph out of the threads data and generate a text representation of the knowledge graph. The summary of the knowledge graph with the most important, central nodes is also derived. This is used to create the Vector DB of the Proposed Arch.

Converting Tweets dataframe into Twitter Threads

In [6]:
import re
import pandas as pd
from google.colab import files

def extract_company_name(text):
    match = re.search(r'@(\w+)', text)
    return match.group(1) if match else None

def convert_dataframe_to_collated_threads(df):
    tweets = {}
    company_threads = {}
    unknown_company_threads = []
    thread_counter = 1

    for _, row in df.iterrows():
        tweet_id = int(row['tweet_id'])
        tweets[tweet_id] = {
            'tweet_id': tweet_id,
            'author_id': row['author_id'],
            'text': row['text'],
            'in_response_to_tweet_id': float(row['in_response_to_tweet_id']) if pd.notna(row['in_response_to_tweet_id']) else None,
            'response_tweet_id': row['response_tweet_id']
        }

    root_tweets = [tweet for tweet in tweets.values() if tweet['in_response_to_tweet_id'] is None]

    for root_tweet in root_tweets:
        thread_output = f'"thread_id": {thread_counter}\n'
        company_name = extract_company_name(root_tweet['text'])
        if company_name:
            thread_output += f'"company_name": {company_name}\n'

        thread_tweets = []
        current_tweet = root_tweet
        while current_tweet:
            thread_tweets.append(current_tweet)
            response_ids = str(current_tweet['response_tweet_id']).split(',') if pd.notna(current_tweet['response_tweet_id']) else []
            next_tweet = None
            for response_id in response_ids:
                if response_id.isdigit() and int(response_id) in tweets:
                    next_tweet = tweets[int(response_id)]
                    break
            current_tweet = next_tweet

        for tweet in thread_tweets:
            tweet_text = f'"author_id": "{tweet["author_id"]}"\n"text": "{tweet["text"]}"\n'
            thread_output += tweet_text

        thread_output += "\n\n"

        if company_name:
            if company_name not in company_threads:
                company_threads[company_name] = []
            company_threads[company_name].append(thread_output)
        else:
            unknown_company_threads.append(thread_output)

        thread_counter += 1

    output = ""
    for company, threads in company_threads.items():
        company_section = f"Company: {company}\n\n" + "".join(threads)
        output += company_section

    if unknown_company_threads:
        unknown_section = "Unknown Companies\n\n" + "".join(unknown_company_threads)
        output += unknown_section

    return output



text_output = convert_dataframe_to_collated_threads(complete_tweets_dataframe)

print(f"Total characters: {len(text_output)}")

print("\nFirst 100 characters of the text output:")
print(text_output[:100])

# Save the output to a file and initiate download
output_file_name = 'collated_threads.txt'
with open(output_file_name, 'w', encoding='utf-8') as f:
    f.write(text_output)

files.download(output_file_name)

print(f"\nText has been saved as {output_file_name} and download initiated.")

# Path to the file in Colab
colab_file_path = f"/content/{output_file_name}"
print(f"\nThe file is now available in your Colab environment at: {colab_file_path}")
print("You can use this path to read the file in subsequent cells.")

Total characters: 913804

First 100 characters of the text output:
Company: sprintcare

"thread_id": 1
"company_name": sprintcare
"author_id": "115712"
"text": "@sprin


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Text has been saved as collated_threads.txt and download initiated.

The file is now available in your Colab environment at: /content/collated_threads.txt
You can use this path to read the file in subsequent cells.


# Creating Knowledge Graph

The knowledge graph is created using the twitter thread dataset in text form. The SpaCy library is used for NLP on the text and then valid nodes are filtered out to generate the Knowledge graph.

In [19]:
import spacy
import networkx as nx
import plotly.graph_objects as go
import re

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    text = re.sub(r'[^\w\s@#]', '', text)
    return text.lower().strip()

def is_valid_entity(text):
    if not text or text.isspace():
        return False
    filler_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it',
                    'its', 'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'we',
                    'they', 'them', 'my', 'but', 'have', 'has', 'had', 'do', 'does', 'did', 'can', 'could', 'would',
                    'should', 'this', 'these', 'those', 'there', 'here', 'where', 'when', 'why', 'how', 'what', 'who'}
    if text in filler_words or text.isdigit():
        return False
    return True

def is_customer_support_term(term):
    support_terms = {'help', 'support', 'assist', 'resolve', 'issue', 'problem', 'question', 'concern', 'complaint',
                     'service', 'account', 'bill', 'payment', 'refund', 'cancel', 'upgrade', 'downgrade', 'technical',
                     'error', 'bug', 'outage', 'down', 'slow', 'fast', 'connection', 'speed', 'customer', 'representative'}
    return term in support_terms

def create_knowledge_graph(text):
    tweets = re.split(r'"thread_id":', text)[1:]
    G = nx.Graph()

    for tweet in tweets:
        company_match = re.search(r'"company_name": (\w+)', tweet)
        text_match = re.search(r'"text": "([^"]+)"', tweet)

        if company_match and text_match:
            company = company_match.group(1)
            tweet_text = text_match.group(1)

            if is_valid_entity(company):
                G.add_node(company, type='company')

            doc = nlp(tweet_text)

            for token in doc:
                if is_valid_entity(token.text) and (is_customer_support_term(token.lemma_) or token.ent_type_):
                    clean_text = preprocess_text(token.text)
                    if is_valid_entity(clean_text):
                        G.add_node(clean_text, type='term')
                        if is_valid_entity(company):
                            G.add_edge(company, clean_text)

            hashtags = re.findall(r'#(\w+)', tweet_text)
            for hashtag in hashtags:
                if is_valid_entity(hashtag):
                    G.add_node(hashtag, type='hashtag')
                    if is_valid_entity(company):
                        G.add_edge(company, hashtag)

    return G

def improve_graph(G, top_n=100):
    degree_dict = dict(G.degree())
    sorted_nodes = sorted(degree_dict, key=degree_dict.get, reverse=True)
    top_nodes = sorted_nodes[:top_n]
    G_filtered = G.subgraph(top_nodes)
    centrality = nx.betweenness_centrality(G_filtered)
    return G_filtered, centrality

def visualize_improved_graph(G, centrality):
    pos = nx.spring_layout(G, k=0.5, iterations=50)

    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='#888'), hoverinfo='none', mode='lines')

    node_x, node_y = [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='Viridis',
            size=10,
            colorbar=dict(thickness=15, title='Node Centrality', xanchor='left', titleside='right')
        )
    )

    node_centrality = list(centrality.values())
    node_text = [f'{node}<br>Centrality: {centrality[node]:.4f}' for node in G.nodes()]

    node_trace.marker.color = node_centrality
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Customer Support Knowledge Graph',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )

    fig.show()

# Usage
with open('/content/collated_threads.txt', 'r', encoding='utf-8') as file:
    collated_threads_text = file.read()

G = create_knowledge_graph(collated_threads_text)

if G.number_of_nodes() > 0:
    print("\nSample of nodes:")
    for node in list(G.nodes())[:20]:
        print(node)

    G_filtered, centrality = improve_graph(G)
    visualize_improved_graph(G_filtered, centrality)
else:
    print("No nodes were created in the graph.")

Number of nodes in the graph: 1619
Number of edges in the graph: 1627

Sample of nodes:
sprintcare
customer
service
slow
help
@115714
upgrading
outage
minneapolis
st
paul
minnesota
connection
lte
support
account
years
spring
corporate
two


Summarised Knowledge Graph

In [22]:
import networkx as nx
from google.colab import files

def graph_to_text(G):
    lines = []

    # Add nodes and their attributes
    lines.append("NODES:")
    for node, data in G.nodes(data=True):
        attr_str = "|".join(f"{k}:{v}" for k, v in data.items())
        lines.append(f"{node}\t{attr_str}")

    # Add edges and their attributes
    lines.append("\nEDGES:")
    for u, v, data in G.edges(data=True):
        attr_str = "|".join(f"{k}:{v}" for k, v in data.items())
        lines.append(f"{u}\t{v}\t{attr_str}")

    return "\n".join(lines)

def export_graph_as_text(G, filename='twitter_customer_service_graph_metadata.txt'):
    text_representation = graph_to_text(G)

    return text_representation


# Usage
knowledge_graph_summary = export_graph_as_text(G)

Get Summary and high Priority Nodes

In [25]:

from collections import Counter


def generate_metadata_summary(G):
    summary = []

    # Total nodes and edges
    summary.append(f"Total nodes in the graph: {G.number_of_nodes()}")
    summary.append(f"Total edges in the graph: {G.number_of_edges()}")

    # Company statistics
    companies = [node for node, data in G.nodes(data=True) if data.get('type') == 'company']
    summary.append(f"\nNumber of unique companies: {len(companies)}")
    summary.append("Top 5 companies by connections:")
    company_degrees = sorted([(company, G.degree(company)) for company in companies], key=lambda x: x[1], reverse=True)[:5]
    for company, degree in company_degrees:
        summary.append(f"  - {company}: {degree} connections")

    # Customer support term statistics
    terms = [node for node, data in G.nodes(data=True) if data.get('type') == 'term']
    summary.append(f"\nNumber of unique customer support terms: {len(terms)}")
    summary.append("Top 10 customer support terms by connections:")
    term_degrees = sorted([(term, G.degree(term)) for term in terms], key=lambda x: x[1], reverse=True)[:10]
    for term, degree in term_degrees:
        summary.append(f"  - {term}: {degree} connections")

    # Hashtag statistics
    hashtags = [node for node, data in G.nodes(data=True) if data.get('type') == 'hashtag']
    summary.append(f"\nNumber of unique hashtags: {len(hashtags)}")
    summary.append("Top 5 hashtags by connections:")
    hashtag_degrees = sorted([(hashtag, G.degree(hashtag)) for hashtag in hashtags], key=lambda x: x[1], reverse=True)[:5]
    for hashtag, degree in hashtag_degrees:
        summary.append(f"  - {hashtag}: {degree} connections")

    # Graph structure statistics
    summary.append(f"\nGraph density: {nx.density(G):.4f}")
    try:
        summary.append(f"Graph diameter: {nx.diameter(G)}")
    except nx.NetworkXError:
        summary.append("Graph diameter: Not applicable (graph is not connected)")

    summary.append(f"\nTop 5 nodes by betweenness centrality:")
    betweenness = nx.betweenness_centrality(G)
    top_betweenness = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)[:5]
    for node, centrality in top_betweenness:
        summary.append(f"  - {node}: {centrality:.4f}")

    return "\n".join(summary)

# Usage
with open('/content/collated_threads.txt', 'r', encoding='utf-8') as file:
    text = file.read()


if G.number_of_nodes() > 0:
    # Generate and print metadata summary
    metadata_summary = generate_metadata_summary(G)
else:
    print("No nodes were created in the graph.")

# Proposed Arch Vector DB

In [37]:
import os
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import numpy as np

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize Pinecone
pc = Pinecone(api_key="93ed05c3-1091-4c1b-b734-f2976eee3cd4")

index_name = "vector-db-proposed-arch"

# Create Pinecone index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine"
    )

# Connect to the index
index = pc.Index(index_name)

def preprocess_text(text, chunk_size=1000):
    """Split text into chunks."""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def embed_and_upload(text, metadata, prefix):
    """Embed text chunks and upload to Pinecone."""
    chunks = preprocess_text(text)
    for i, chunk in enumerate(chunks):
        embedding = model.encode(chunk).tolist()
        index.upsert(vectors=[(f"{prefix}-{i}", embedding, {**metadata, "chunk": i, "text": chunk})])

# 1. Process Twitter threads data
with open('/content/collated_threads.txt', 'r', encoding='utf-8') as file:
    twitter_threads = file.read()

embed_and_upload(twitter_threads, {"type": "twitter_threads"}, "threads")

# 2. Process knowledge graph details
knowledge_graph_details = export_graph_as_text(G)
embed_and_upload(knowledge_graph_details, {"type": "knowledge_graph"}, "graph")

# 3. Process metadata summary
with open('metadata_summary.txt', 'r', encoding='utf-8') as file:
    metadata_summary = file.read()

embed_and_upload(metadata_summary, {"type": "metadata_summary"}, "summary")

print("Vector database created and populated.")




Vector database created and populated.


In [40]:
def query_vector_db(query_text, top_k=5):
    """
    Query the vector database and return top k results as concatenated text.

    :param query_text: The text to query the vector database
    :param top_k: The number of top results to return
    :return: Concatenated text from the top k results
    """
    query_embedding = model.encode(query_text).tolist()
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    results_text = f"Top {top_k} results for query: '{query_text}':\n\n"
    for result in results['matches']:
        results_text += f"Score: {result['score']:.4f}\n"
        results_text += f"Type: {result['metadata']['type']}\n"
        results_text += f"Text: {result['metadata']['text'][:200]}...\n\n"  # Append first 200 characters
    return results_text


## Calling Falcon Model

In [58]:
from ai71 import AI71
import os

# Set your AI71 API key
AI71_API_KEY = os.environ.get("AI71_API_KEY", "api71-api-524a603b-3fc5-4ae1-955f-3c7c2e636fc5")

# Initialize the AI71 client
client = AI71(AI71_API_KEY)

def send_prompt_to_falcon(static_instructions, vector_db_query, user_prompt, conversation_history, model="tiiuae/falcon-180B-chat"):
    """
    Send a prompt to the Falcon model using AI71 API and return the response.

    :param static_instructions: Static set of instructions for the model
    :param vector_db_query: Query result from the vector database
    :param user_prompt: The user's prompt
    :param conversation_history: The history of the conversation so far
    :param model: The specific Falcon model to use (default is falcon-180B-chat)
    :return: The model's response as a string
    """
    try:
        messages = []
       # Append conversation history
        for history in conversation_history:
            messages.append(history)

        # Combine all parts into a single prompt
        messages = [
            {"role": "system", "content": static_instructions},
            {"role": "system", "content": f"Relevant information from database: {vector_db_query}"},
            {"role": "user", "content": user_prompt},
        ]

        # Create a chat completion
        response = client.chat.completions.create(
            model=model,
            messages=messages,
        )

        # Extract and return the model's response
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {str(e)}"


# Testing

In [75]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import re

# Download necessary NLTK data
nltk.download('vader_lexicon')

# Create the dictionary of test cases (same as before)
test_cases = {
    'Case 1': {
        'user_message': "@Microsoft I can't log into my account. Help!",
        'expected_response': "I'm sorry to hear you're having trouble logging in. Can you please DM us with your account email address? We'll help you regain access.",
        'user_followup': "Sent a DM.",
        'expected_followup_response': "Thank you for sending a DM. We've received it and a support agent will assist you shortly with regaining access to your account."
    },
    'Case 2': {
        'user_message': "@Sprintcare Why is my bill so high this month?",
        'expected_response': "I understand your concern about your bill. For your privacy, please DM us your account number and we'll review your charges in detail.",
        'user_followup': "I don't want to DM. Can't you just tell me here?",
        'expected_followup_response': "We take your privacy seriously and can't discuss specific account details publicly. DMing us is the quickest way to get a thorough explanation of your bill. Is there a general billing question I can help with instead?"
    }
}
# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(test_cases, orient='index')

def calculate_appropriateness(actual, expected):
    # Check if key phrases are present
    key_phrases = ["DM", "direct message", "privacy", "account", "help", "assist"]
    actual_lower = actual.lower()
    score = sum(phrase in actual_lower for phrase in key_phrases) / len(key_phrases)
    return int(score * 40)  # 0-40 points

def calculate_helpfulness(actual, expected):
    # Check for action items or next steps
    action_phrases = ["please", "can you", "we'll", "we will", "next step"]
    actual_lower = actual.lower()
    action_score = sum(phrase in actual_lower for phrase in action_phrases) / len(action_phrases)

    # Check for specific information provided
    info_score = len(re.findall(r'\d+', actual)) / max(len(re.findall(r'\d+', expected)), 1)

    return int((action_score * 0.7 + info_score * 0.3) * 30)  # 0-30 points

def calculate_tone(text):
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(text)

    # We want a positive but not overly positive tone
    if sentiment_scores['compound'] > 0.5:
        return 25  # Very positive
    elif sentiment_scores['compound'] > 0:
        return 30  # Moderately positive (ideal for customer service)
    elif sentiment_scores['compound'] > -0.5:
        return 20  # Neutral
    else:
        return 10  # Negative

def calculate_similarity(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    cosine_sim = cosine_similarity(vectorizer)
    return cosine_sim[0][1]

def calculate_scores(actual_response, expected_response):
    appropriateness = calculate_appropriateness(actual_response, expected_response)
    helpfulness = calculate_helpfulness(actual_response, expected_response)
    tone = calculate_tone(actual_response)

    # Use similarity as a general check
    similarity = calculate_similarity(actual_response, expected_response)

    # Adjust scores based on overall similarity
    appropriateness = int(appropriateness * (0.7 + 0.3 * similarity))
    helpfulness = int(helpfulness * (0.7 + 0.3 * similarity))

    total_score = appropriateness + helpfulness + tone

    return {
        'Appropriateness': appropriateness,
        'Helpfulness': helpfulness,
        'Tone': tone,
        'Total Score': total_score
    }

# Function to process responses and calculate scores (same as before)
def process_responses(df, actual_responses):
    results = []
    for index, row in df.iterrows():
        # Initial response
        initial_scores = calculate_scores(actual_responses[index]['initial'], row['expected_response'])
        initial_scores['Case'] = index
        initial_scores['Response Type'] = 'Initial'
        results.append(initial_scores)

        # Follow-up response
        followup_scores = calculate_scores(actual_responses[index]['followup'], row['expected_followup_response'])
        followup_scores['Case'] = index
        followup_scores['Response Type'] = 'Follow-up'
        results.append(followup_scores)

    return pd.DataFrame(results)

# Example usage:


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Proposed Arch Responses

In [62]:
if __name__ == "__main__":
    static_instructions = "You are a customer service assistant on Twitter. Provide clear, concise, approprite, and helpful responses in the right tone to user queries. Always aim to resolve the user's issue in a polite and professional manner. Only provide one response to each query and only provide responses as the customer service assistant."
    user_prompt = "I don't want to DM. Can't you just tell me here?"
    vector_db_query = query_vector_db(user_prompt)
    conversation_history = [{"role": "user", "content": "@Sprintcare Why is my bill so high this month?"},
        {"role": "assistant", "content": "I'm sorry to hear that you're experiencing an issue with your bill. Please send us a direct message with your account information and we'll be happy to take a look and provide you with an explanation. Thank you for bringing this to our attention."
}]

    print("Sending prompt to Falcon model...")
    response = send_prompt_to_falcon(static_instructions, vector_db_query, user_prompt, conversation_history)

    print(response)

Sending prompt to Falcon model...
 I'm sorry, but I'm not able to provide you with the information you're looking for via Twitter. However, I'd be happy to assist you if you could please provide me with more details about your query.


In [76]:
actual_responses = {
    'Case 1': {
        'initial': "Hi there! I'm sorry to hear that you're having trouble logging into your account. Can you please provide me with more details about the issue you're experiencing? Are you receiving any error messages? #MicrosoftSupport",
        'followup': "Thank you for reaching out to us. We have received your DM and will respond to it as soon as possible. Please allow us some time to review your message and get back to you with a resolution. Thank you for your patience."
    },
    'Case 2': {
        'initial': "I'm sorry to hear that you're experiencing an issue with your bill. Please send us a direct message with your account information and we'll be happy to take a look and provide you with an explanation. Thank you for bringing this to our attention.",
        'followup': "I'm sorry, but I'm not able to provide you with the information you're looking for via Twitter. However, I'd be happy to assist you if you could please provide me with more details about your query."
    },
}

# Process responses and get scores
scores_df = process_responses(df, actual_responses)

# Calculate average score
average_score = scores_df['Total Score'].mean()

print(scores_df)
print(f"\nAverage Score: {average_score:.2f}")

   Appropriateness  Helpfulness  Tone  Total Score    Case Response Type
0                5            6    10           21  Case 1       Initial
1                0            3    25           28  Case 1     Follow-up
2               10            6    25           41  Case 2       Initial
3                4            2    25           31  Case 2     Follow-up

Average Score: 30.25
